In [1]:
import pandas as pd
import numpy as np

DATASETS_LOCATION = "/Users/gomerudo/workspace/datasets"

## Load clean datasets

In [2]:
# Loading from absolute path, because the dataset is too big to download and to upload

DATASET_CRIMES_PATH = DATASETS_LOCATION + "/chicago_crimes_2001_present_clean.csv"
DATASET_INDICATORS_PATH = DATASETS_LOCATION + "/chicago_socioeconomic_indicators_2008_2012_clean.csv"

# Load CSVs
crimes_df = pd.read_csv(DATASET_CRIMES_PATH, na_values = ["", " "])
indicators_df = pd.read_csv(DATASET_INDICATORS_PATH, na_values = ["", " "])

In [3]:
# Show preview of crimes
crimes_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Community Area,Year,Latitude,Longitude
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,1111,11,25,2015,41.891399,-87.744385
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,725,7,67,2015,41.773372,-87.665319
2,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,222,2,39,2015,41.813861,-87.596643
3,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,225,2,40,2015,41.800802,-87.622619
4,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,1113,11,25,2015,41.878065,-87.743354


In [4]:
# Show preview of indicators
indicators_df.head()

,Community Area Number,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,39.000000,4.923377,21.766234,15.372727,20.341558,35.746753,25563.168831,49.506494
std,22.371857,3.682887,11.529957,7.543412,11.823152,7.327716,15293.098259,28.690556
min,1.000000,0.300000,3.300000,4.700000,2.500000,13.500000,8201.000000,1.000000
25%,20.000000,2.300000,13.200000,9.200000,11.800000,32.000000,15754.000000,25.000000
50%,39.000000,3.800000,18.900000,13.900000,18.500000,38.100000,21323.000000,50.000000
75%,58.000000,6.800000,29.200000,20.000000,26.600000,40.500000,28887.000000,74.000000
max,77.000000,15.800000,56.500000,35.900000,54.800000,51.500000,88669.000000,98.000000


## Do the stuff for the network creation :D

### - First create the csv for the nodes

In [6]:
import time

start_time = time.time()

nodes_list = []

headers = ["NodeID", 
           "Property1", "Value1", 
           "Property2", "Value2", 
           "Property3", "Value3", 
           "Property4", "Value4",
           "Property5", "Value5"]

# For communities
communities = crimes_df['Community Area'].unique()
communities = np.sort(communities)

for community in communities:
    nodes_list.append(["C{}".format(community), # NodeID
                       'NodeLabel', # Property1
                       'Community', # Value1
                       'Name', # Property2
                       "Community {}".format(community), # Value2
                       np.NAN, # Property3
                       np.NAN, # Value3
                       np.NAN, # Property4
                       np.NAN, # Value4
                       np.NAN, # Property5
                       np.NAN]) # Value5

# For districts
districts = crimes_df['District'].unique()
districts = np.sort(districts)

for district in districts:
    nodes_list.append(["D{}".format(district), #NodeID
                       'NodeLabel', # Property1
                       'District', # Value1
                       'Name', # Property2
                       "District {}".format(district), # Value2
                       np.NAN, # Property3
                       np.NAN, # Value3
                       np.NAN, # Property4
                       np.NAN, # Value4
                       np.NAN, # Property5
                       np.NAN]) # Value5

# For primary type...
crime_types = crimes_df['Primary Type'].unique()
crime_types = np.sort(crime_types)

# for i, crime_type in enumerate(crime_types):
#     nodes_list.append(["CT_{}".format(i), # NodeID
#                        'NodeLabel', # Property1
#                        'CrimeType', # Value1
#                        'Name', # Property2
#                        crime_type, # Value2
#                        '', # Property 3
#                        '']) # Value3

# For case numbers
for index, row in crimes_df.iterrows():
    nodes_list.append(["CASE_{}".format(row['Case Number']), # NodeID
                       'NodeLabel', # Property1
                       'CaseNumber', # Value1
                       'Name', # Property2
                       row['Case Number'], # Value2
                       'Type', # Property3
                       row['Primary Type'], # Value3
                       'Latitude', # Property4
                       row['Latitude'], # Value4
                       'Longitude', # Property5
                       row['Longitude'] ]) # Value5
    
# cases = crimes_df['Case Number'].unique()
# cases = np.sort(cases)

# for case in cases:
#     nodes_list.append(["CASE_{}".format(case) , 
#                        'NodeLabel', 
#                        'CaseNumber', 
#                        'Name', 
#                        case,
#                        ''])

# Create other nodes (based on criterias of indicators and so on)
# Based on hardship
nodes_list.append(["CR_DAN".format(), # NodeID
                   'NodeLabel', # Property1
                   'Category', # Value1
                   'Name', # Property2
                   'Dangereous', # Value2
                   'Comment', # Property3
                   'Bassed on hardship index', # Value3
                   np.NAN, # Property4
                   np.NAN, # Value4
                   np.NAN, # Property5
                   np.NAN]) # Value5

# Create the pandas DataFrame to save the results
nodes_df = pd.DataFrame(nodes_list, columns = headers)

end_time = time.time()
print("Creation of nodes data frame took {:.2f} sec".format(end_time - start_time))

nodes_df

Creation of nodes data frame took 937.41 sec


,NodeID,Property1,Value1,Property2,Value2,Property3,Value3,Property4,Value4,Property5,Value5
0,C1,NodeLabel,Community,Name,Community 1,NaN,NaN,NaN,NaN,NaN,NaN
1,C2,NodeLabel,Community,Name,Community 2,NaN,NaN,NaN,NaN,NaN,NaN
2,C3,NodeLabel,Community,Name,Community 3,NaN,NaN,NaN,NaN,NaN,NaN
3,C4,NodeLabel,Community,Name,Community 4,NaN,NaN,NaN,NaN,NaN,NaN
4,C5,NodeLabel,Community,Name,Community 5,NaN,NaN,NaN,NaN,NaN,NaN
5,C6,NodeLabel,Community,Name,Community 6,NaN,NaN,NaN,NaN,NaN,NaN
6,C7,NodeLabel,Community,Name,Community 7,NaN,NaN,NaN,NaN,NaN,NaN
7,C8,NodeLabel,Community,Name,Community 8,NaN,NaN,NaN,NaN,NaN,NaN
8,C9,NodeLabel,Community,Name,Community 9,NaN,NaN,NaN,NaN,NaN,NaN
9,C10,NodeLabel,Community,Name,Community 10,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Save nodes for further usage
DATASET_NODES_PATH = DATASETS_LOCATION + "/timewise_nodes.csv"
nodes_df.to_csv(DATASET_NODES_PATH, index = False)

### - Now, for the edges...

#### Relate communties with cases

In theory, this would never change, so we shouldn't modify this cell too much...

In [55]:
import time

# Log time
start_time = time.time()

# Array to store the edges
edges_list = []

headers = ["EdgeID", "FromNodeID", "ToNodeID",
           "Property1", "Value1", 
           "Property2", "Value2", 
           "Property3", "Value3"]

# Get the communities satisfying a given criteria
communities_criteria = indicators_df[indicators_df['HARDSHIP INDEX'] > 50]

# We iterate over the whole dataset to associate all the cases.
for index, row in crimes_df.iterrows():
    if index % 500000 == 0:
        print("Reached row {} ...".format(index))
    edges_list.append(
        [
            "OCURRED_IN", # EdgeID
            "CASE_" + row['Case Number'], # FromNodeID (A case)
            "C" + str(row['Community Area']), # ToNodeID (A community)
            "StartTime", # Property1
            row['Date'], # Value1. TODO: save start of period we should define
            "EndTime", # Property2
            row['Date'], # Value2. TODO: save end of period we should define
            "Timestamp", # Property3
            row['Date'] # Value3
        ]
    )

pd.DataFrame(edges_list, columns = headers)

edges_df = pd.DataFrame(edges_list, columns = headers)

end_time = time.time()
print("Creation of edges data frame took {:.2f} sec".format(end_time - start_time))

edges_df

Reached row 0 ...
Reached row 500000 ...
Reached row 1000000 ...
Reached row 1500000 ...
Reached row 2000000 ...
Reached row 2500000 ...
Reached row 3000000 ...
Reached row 3500000 ...
Reached row 4000000 ...
Reached row 4500000 ...
Reached row 5000000 ...
Reached row 5500000 ...
Creation of nodes data frame took 768.75 sec


,EdgeID,FromNodeID,ToNodeID,Property1,Value1,Property2,Value2,Property3,Value3
0,OCURRED_IN,CASE_HY189866,C25,StartTime,03/18/2015 07:44:00 PM,EndTime,03/18/2015 07:44:00 PM,Timestamp,03/18/2015 07:44:00 PM
1,OCURRED_IN,CASE_HY190059,C67,StartTime,03/18/2015 11:00:00 PM,EndTime,03/18/2015 11:00:00 PM,Timestamp,03/18/2015 11:00:00 PM
2,OCURRED_IN,CASE_HY190052,C39,StartTime,03/18/2015 10:45:00 PM,EndTime,03/18/2015 10:45:00 PM,Timestamp,03/18/2015 10:45:00 PM
3,OCURRED_IN,CASE_HY190054,C40,StartTime,03/18/2015 10:30:00 PM,EndTime,03/18/2015 10:30:00 PM,Timestamp,03/18/2015 10:30:00 PM
4,OCURRED_IN,CASE_HY189976,C25,StartTime,03/18/2015 09:00:00 PM,EndTime,03/18/2015 09:00:00 PM,Timestamp,03/18/2015 09:00:00 PM
5,OCURRED_IN,CASE_HY190032,C39,StartTime,03/18/2015 10:00:00 PM,EndTime,03/18/2015 10:00:00 PM,Timestamp,03/18/2015 10:00:00 PM
6,OCURRED_IN,CASE_HY190047,C68,StartTime,03/18/2015 11:00:00 PM,EndTime,03/18/2015 11:00:00 PM,Timestamp,03/18/2015 11:00:00 PM
7,OCURRED_IN,CASE_HY189988,C38,StartTime,03/18/2015 09:35:00 PM,EndTime,03/18/2015 09:35:00 PM,Timestamp,03/18/2015 09:35:00 PM
8,OCURRED_IN,CASE_HY190020,C59,StartTime,03/18/2015 10:09:00 PM,EndTime,03/18/2015 10:09:00 PM,Timestamp,03/18/2015 10:09:00 PM
9,OCURRED_IN,CASE_HY189964,C49,StartTime,03/18/2015 09:25:00 PM,EndTime,03/18/2015 09:25:00 PM,Timestamp,03/18/2015 09:25:00 PM


#### Relate communities with categories (dangereous, poor, etc...)

In [56]:
# This cell appends the content. So, if you run it multiple times, you will add multiple times.
# Proceed carefully...

import time

# Log time
start_time = time.time()

# Get the communities satisfying a given criteria
communities_criteria = indicators_df[indicators_df['HARDSHIP INDEX'] > 50]

for index, row in communities_criteria.iterrows():
    edges_list.append(
        [
            "CONSIDERED", # EdgeID
            "C" + str(row['Community Area Number']), # FromNodeID (A community)
            "CR_DAN", # ToNodeID (A criteria node)
            "HardshipIndex", # Property1
            row['HARDSHIP INDEX'], # Value1
            np.NaN, # Property2
            np.NaN, # Value2
            np.NaN, # Property3
            np.NaN  # Value3
        ]
    )

edges_df = pd.DataFrame(edges_list, columns = headers)

end_time = time.time()
print("Append to edges data frame took {:.2f} sec".format(end_time - start_time))

edges_df
    

Creation of nodes data frame took 9.33 sec


,EdgeID,FromNodeID,ToNodeID,Property1,Value1,Property2,Value2,Property3,Value3
0,OCURRED_IN,CASE_HY189866,C25,StartTime,03/18/2015 07:44:00 PM,EndTime,03/18/2015 07:44:00 PM,Timestamp,03/18/2015 07:44:00 PM
1,OCURRED_IN,CASE_HY190059,C67,StartTime,03/18/2015 11:00:00 PM,EndTime,03/18/2015 11:00:00 PM,Timestamp,03/18/2015 11:00:00 PM
2,OCURRED_IN,CASE_HY190052,C39,StartTime,03/18/2015 10:45:00 PM,EndTime,03/18/2015 10:45:00 PM,Timestamp,03/18/2015 10:45:00 PM
3,OCURRED_IN,CASE_HY190054,C40,StartTime,03/18/2015 10:30:00 PM,EndTime,03/18/2015 10:30:00 PM,Timestamp,03/18/2015 10:30:00 PM
4,OCURRED_IN,CASE_HY189976,C25,StartTime,03/18/2015 09:00:00 PM,EndTime,03/18/2015 09:00:00 PM,Timestamp,03/18/2015 09:00:00 PM
5,OCURRED_IN,CASE_HY190032,C39,StartTime,03/18/2015 10:00:00 PM,EndTime,03/18/2015 10:00:00 PM,Timestamp,03/18/2015 10:00:00 PM
6,OCURRED_IN,CASE_HY190047,C68,StartTime,03/18/2015 11:00:00 PM,EndTime,03/18/2015 11:00:00 PM,Timestamp,03/18/2015 11:00:00 PM
7,OCURRED_IN,CASE_HY189988,C38,StartTime,03/18/2015 09:35:00 PM,EndTime,03/18/2015 09:35:00 PM,Timestamp,03/18/2015 09:35:00 PM
8,OCURRED_IN,CASE_HY190020,C59,StartTime,03/18/2015 10:09:00 PM,EndTime,03/18/2015 10:09:00 PM,Timestamp,03/18/2015 10:09:00 PM
9,OCURRED_IN,CASE_HY189964,C49,StartTime,03/18/2015 09:25:00 PM,EndTime,03/18/2015 09:25:00 PM,Timestamp,03/18/2015 09:25:00 PM


In [58]:
# Save nodes for further usage
DATASET_NODES_PATH = DATASETS_LOCATION + "/timewise_edges.csv"
edges_df.to_csv(DATASET_NODES_PATH, index = False)